In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from GANs.train import GAN
import pytorch_lightning as pl

import numpy as np

from torchvision import transforms

In [3]:
loss_type = 'standard'

tr = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.52, ), (0.12, )),
#     transforms.RandomHorizontalFlip(),
#     transforms.RandomVerticalFlip(),
#     transforms.RandomRotation([-5, 5])
])

In [4]:
def train(experiment_name, use_gpu):
    assignment_dir = '../'

    logger = pl.loggers.TensorBoardLogger(save_dir=f'{assignment_dir}/logs', name=experiment_name)

    trainer = pl.Trainer(
        max_epochs=250, 
        gpus=[1, 2, 3] if use_gpu else None,
        num_sanity_val_steps=2,
        benchmark=True, 
        log_every_n_steps=5,
        check_val_every_n_epoch=2,
        gradient_clip_val=0.5,
#         limit_train_batches=25,
        strategy='dp',
        logger=logger) 
    
    return trainer

In [5]:
gan = GAN(train_data_path = "../data/dataset_train.npy", 
          valid_data_path = "../data/dataset_test.npy",
          gan_type='dcgan', #biggan
          batch_size=24, 
          loss_type=loss_type, 
          transforms=tr)

# gan = GAN.load_from_checkpoint(f"{loss_type}_loss_zone_0_1_3.ckpt",
#                          train_data_path = "./data/dataset_full.npy", 
#                          valid_data_path = "./data/dataset_zone5.npy",
#                          batch_size=12, 
#                          loss_type=loss_type, 
#                          transforms=tr)

In [6]:
trainer = train(f"{loss_type}_loss_zone_0_1_3", True)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(gan)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name    | Type          | Params
------------------------------------------
0 | loss    | GanLoss       | 0     
1 | encoder | Encoder       | 11.7 M
2 | gen     | Generator     | 1.7 M 
3 | dis     | Discriminator | 901 K 
------------------------------------------
14.3 M    Trainable params
0         Non-trainable params
14.3 M    Total params
57.303    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

In [ ]:
trainer.save_checkpoint(f"{loss_type}_loss_zone_0_1_3_biggan.ckpt")